In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf


2022-06-25 11:52:26.107735: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-25 11:52:26.107762: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import BinaryCrossentropy

In [3]:
train = pd.read_csv('../titanic-raw/train.csv')
test = pd.read_csv('../titanic-raw/test.csv')


In [4]:
train.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [6]:
train.shape , test.shape

((891, 12), (418, 11))

In [7]:
print(list(train.columns))

['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']


In [8]:
train.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [9]:
train.describe()


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [10]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [11]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [12]:
#fixing train_data

train2 = train.copy()
train2['Age'] = train2['Age'].replace(np.NAN, train2['Age'].mean())
train['Age'].mean() , train2['Age'].mean()
train2['Cabin'] = train2['Cabin'].replace(np.NAN, 'XX')
train2['Cabin'].value_counts()
train2['Embarked'] = train2['Embarked'].replace(np.NAN, 'YY')
train2['Embarked'].value_counts()
train2.isnull().sum()


PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [13]:
#cleaning test data

test2 = test.copy()
test2['Age'] = test2['Age'].replace(np.NAN, test2['Age'].mean())
test['Age'].mean() , test2['Age'].mean()
test2['Cabin'] = test2['Cabin'].replace(np.NAN, 'XX')
test2['Cabin'].value_counts(dropna = False)
test2['Fare'] = test2['Fare'].replace(np.NAN, test2['Fare'].mean())
test['Fare'].mean() , test2['Fare'].mean()
test2.isnull().sum()

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [14]:
trainLM = train2.copy()
testLM = test2.copy()

In [15]:
trainLM.Sex.value_counts()

m1 = {'male':0, 'female':1}
trainLM['Sex'] = trainLM['Sex'].map(m1)
testLM['Sex'] = testLM['Sex'].map(m1) 

# note : running this snippet 2nd time creates NaN value
print(trainLM['Sex'].value_counts())
testLM['Sex'].value_counts()
trainLM.dtypes
testLM.dtypes
trainLM2 = trainLM.iloc[:, [1,2,4,5,6,7,9,]]
print(trainLM2)

0    577
1    314
Name: Sex, dtype: int64
     Survived  Pclass  Sex        Age  SibSp  Parch     Fare
0           0       3    0  22.000000      1      0   7.2500
1           1       1    1  38.000000      1      0  71.2833
2           1       3    1  26.000000      0      0   7.9250
3           1       1    1  35.000000      1      0  53.1000
4           0       3    0  35.000000      0      0   8.0500
..        ...     ...  ...        ...    ...    ...      ...
886         0       2    0  27.000000      0      0  13.0000
887         1       1    1  19.000000      0      0  30.0000
888         0       3    1  29.699118      1      2  23.4500
889         1       1    0  26.000000      0      0  30.0000
890         0       3    0  32.000000      0      0   7.7500

[891 rows x 7 columns]


In [16]:
trainLM2.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
dtype: int64

In [17]:
testLM2 = testLM.iloc[:, [1,3,4,5,6,8]]


In [18]:
#K time to prep train data for neural network
X=tf.convert_to_tensor(trainLM2.drop('Survived',axis=1))
Y=tf.convert_to_tensor(trainLM2['Survived'])


2022-06-25 11:52:27.632366: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-25 11:52:27.632393: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-25 11:52:27.632417: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (CoderCatA5Pop): /proc/driver/nvidia/version does not exist
2022-06-25 11:52:27.632761: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [19]:
type(X)

tensorflow.python.framework.ops.EagerTensor

In [20]:
X

<tf.Tensor: shape=(891, 6), dtype=float64, numpy=
array([[ 3.        ,  0.        , 22.        ,  1.        ,  0.        ,
         7.25      ],
       [ 1.        ,  1.        , 38.        ,  1.        ,  0.        ,
        71.2833    ],
       [ 3.        ,  1.        , 26.        ,  0.        ,  0.        ,
         7.925     ],
       ...,
       [ 3.        ,  1.        , 29.69911765,  1.        ,  2.        ,
        23.45      ],
       [ 1.        ,  0.        , 26.        ,  0.        ,  0.        ,
        30.        ],
       [ 3.        ,  0.        , 32.        ,  0.        ,  0.        ,
         7.75      ]])>

In [21]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(X)


In [22]:
#making our model

model=Sequential([
      normalizer,
      tf.keras.layers.Input(shape=(6,)),
      Dense(units=25,activation="relu"),
      Dense(units=15,activation="relu"),
      Dense(units=5,activation="relu"),
      Dense(units=1,activation="sigmoid"),
      

  ])
model.compile(loss=BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name='accuracy')
    ])


In [23]:
import gc
class EvalCallback(tf.keras.callbacks.Callback):
    def __init__(self):
       pass

    def on_epoch_end(self, epoch: int, logs=None):
        gc.collect()
        tf.keras.backend.clear_session()
    
        

In [24]:
model.fit(X,Y,epochs=100000,verbose=0)

In [25]:
gc.collect()
tf.keras.backend.clear_session()

In [26]:
X_test=tf.convert_to_tensor(testLM2)
Y_pred=model.predict(X_test)


In [27]:
Y_pred.shape

(418, 1)

In [28]:
Y_pred

array([[9.10714269e-03],
       [4.24554012e-12],
       [2.29208469e-02],
       [1.81737930e-01],
       [9.99994576e-01],
       [9.89074111e-02],
       [2.58623242e-01],
       [3.45339208e-15],
       [4.78356898e-01],
       [1.59230828e-03],
       [5.74368536e-02],
       [3.20393652e-01],
       [1.00000000e+00],
       [0.00000000e+00],
       [1.00000000e+00],
       [9.99917626e-01],
       [2.65985847e-01],
       [4.40859497e-02],
       [4.01534089e-06],
       [1.38806291e-08],
       [4.89321066e-15],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.00000000e+00],
       [7.22262549e-30],
       [1.00000000e+00],
       [7.79809356e-02],
       [5.73586822e-01],
       [7.38716424e-02],
       [1.74515357e-38],
       [3.52795448e-32],
       [9.95010138e-04],
       [1.76047683e-02],
       [7.13442028e-01],
       [7.80641735e-02],
       [1.06708825e-01],
       [3.27853531e-01],
       [5.67526519e-02],
       [9.21296954e-01],


In [29]:
A=[]
for i in range(418):
    A.append([int(i)])

In [30]:
A=np.array(A)

In [31]:
A.shape

(418, 1)

In [32]:
Ans=np.stack((A,Y_pred),axis=-1)

In [33]:
np.save("text",Ans)